   # Maxwell's demon with bar magnets

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from numba import jit,njit, prange

%matplotlib widget  
#%matplotlib notebook

## Setting up constants
$B_0$ the strenght of magnetic field.
At this stage, we have to tweek the adjustable constants to achieve the desired motion of the magnets. What I noticed was the resistive force being too high in comparison to fluctuating force. Almost comparable sizes, there might be a few ways to fix this.
### Fixed constants
#### Moment of inertia of a cylinder
http://schoolpress-blogs.myschoolcdn.com/blogs.norfolkacademy.org/sites/11/2018/10/Screen-Shot-2018-10-10-at-10.36.29-PM.png this websites contains the formula, no proof given.

In [3]:
# general constants
d = 10**(-7)       # distance between the two magnets, right now with this separation the magnets would be touching, will change this later
u_0 = 4*np.pi*10**(-7) # vacuum permeability
T = 273.15 + 20    # temperature of the heath bath
B_0 = 0.02         # magnitude of external magnetic field, 1000 Gauss chosen as the highest B field for Fe, at 200 gauss now
k_b = 1.380649*10**(-23)  # Boltzmans constant
nu = 10**(-4)      # 10**(-3)viscosity of water at T=20C, so the medium is 10 times less viscos than water 
E_unit = k_b*T     # kT unit of energy

# system bar magnet
ro_s = 7860          # density of iron
l_s = 5*10**(-7)     # length of the cylinder
R_s = 5*10**(-8)     # radius 
V = np.pi*(R_s**2)*l_s # Volume
M_s = ro_s*V         # mass
I_s = (M_s*R_s**2)/4 + (M_s*l_s**2)/12   # moment of inertia
Mag_s = 0.8*10**(6)  # Magnetization A/m
mu_s = Mag_s*V       # magnetic moment
gam_s = 6*np.pi*nu*(l_s**2)*R_s         # coefficient of friction
b_s = np.sqrt(2*gam_s*k_b*T)    # coefficient b, strenght fluctuating force 

# demon paramagnet
x_d = 7.2*10**(-3) # fe3O4 susceptibility
ro_d = 5000        # density of Fe3O4   
M_d = ro_d*V       # mass
R_d = ( (2/3*V**2) / (np.pi**2) ) **(1/6)         # radius
l_d = V/np.pi * ((2/3*V**2) / (np.pi**2))**(-1/3) # height
I_d = (M_d*R_d**2)/4 + (M_d*l_d**2)/12    # moment of inertia
gam_d = 6*np.pi*nu*(l_d**2)*R_d           # coefficient of friction 
b_d = np.sqrt(2*gam_d*k_b*T)              # coefficient b
Mag_d = 10**4                             # magnetization 
mu_d_max = Mag_d*V                        # maximum magnetic moment

## Adjustable constants
with physical limitations

### Magnetization of system magnet $M_s$
The saturation value of Magnetization for magnets made out of close to pure iron speciments was found to be $1.7x10^6A/m$ which should reach for a magnetic field of about 1000 Gauss(2) (Earths magnetic field ranges from 0.25-0.6 Gauss at the surface). The peak magnetic field of 1000 Gauss doesn't limit us from using more intense fields. Sources: 
1. https://scihubtw.tw/10.1063/1.2163571, https://aip.scitation.org/doi/10.1063/1.2163571 - summary of multiple experimental results from different groups/people
2. https://www.jstor.org/stable/pdf/20025449.pdf - Paper does not include units
3. http://www.brainkart.com/article/Solved-Problems--Magnetic-and-Superconducting-Materials_6825/ - as part of one exercise

Useful website providing conversions between regularly used units in magnetism: https://www.ieeemagnetics.org/index.php?option=com_content&view=article&id=118&Itemid=107

The Currie temperature of Iron is about 1000K so that puts an upper bound on what the temperature of the heath bath could be but we will work with room temperatures anyway. 

Just to start of I'll choose the Magnetization of the system to be half the saturation value, that is probably way too high.

### Magnetization of demon magnet $M_d$
At first I wanted to make the magnet out Fe3o4 material which has a very high susceptibility 7.2x10^-3. This value was taken from EM notes and later I found a same value on http://hyperphysics.phy-astr.gsu.edu/hbase/Tables/magprop.html website that containes susceptibilities of other materials with references. This susceptibility should be valid for room temperatures. 

However in a research paper http://www.ijsei.com/papers/ijsei-10312-13.pdf that shows the relation between magnetization and applied magnetic field H the magnetization M calculated from that graph was very different to the one obtained from susceptibility relation. Not sure what is going on, I might have misread the graph caption and it could display the relation for a different material or at different temperature, or there is an error in some of the calculations or a conceptual flaw in the way I am approaching this. What is even more bizare is that from different papers yielded different magnetization relations. Let's leave this for now and work with a theoretical parameter that fits within a reasonable realistic range of values but can be freely adjusted to our desire. Thus it will not necessarly represent a material or a mixter of compounds that can be found in nature. 

Listing all the relevant papers and website so I can get back to this later.
1. http://www.ijsei.com/papers/ijsei-10312-13.pdf - first paper used to calculate M
2. https://www.researchgate.net/publication/224131637_Magnetic_properties_of_Fe3O4_nanoparticles_coated_with_oleic_and_dodecanoic_acids - another paper with graphs of M/H
3. https://www.sigmaaldrich.com/catalog/product/SAJ/151380?lang=en&region=CZ - website that sells fe3o4 and gives its density
4. https://chem.libretexts.org/Courses/Saint_Marys_College_Notre_Dame_IN/CHEM_431%3A_Inorganic_Chemistry_%28Haas%29/CHEM_431_Readings/14%3A_Magnetism/14.02%3A_Magnetic_Properties_of_Materials - Curie's law, susceptibility dependent on T
5. https://link.springer.com/article/10.1007/s00269-011-0472-x - Currie Temperature of Fe3o4 770-870K
6. https://www.ecosia.org/images?q=magnetization%20curves%20of%20fe3o4%20low%20field#id=9E7345B11C16B6D6A9E06087286E8683FAB1CDA7 - another hysteriosis curves
7. file:///C:/Users/schwarz/Desktop/Magnetic_Vortex_and_Hyperthermia_Suppression_in_Mu.pdf - the one I used in another M calculation
8. http://ferrocell.us/references/Magnetite%20%28Fe3O4%29_%20Properties%20Synthesis%20and%20Applications.pdf - Document about Fe3o4, its physical properties etc.. contains a lot of info


### Drag coefficient $\gamma$
As of now I have not been able to find a precise expresion for what the drag coefficient on a rotating cylinder should. I have spend quite a considerable time on this but now I really need to move forward and make some progress. If needed I can come back to finding the appropriate coefficient later, for now the group's coefficient will be used, I have contacted them asking about its origin but they did not get to me yet. 
The drag coefficient that they used was
$$\beta=6\pi\mu L^3$$

This expression if valid is applicable for needles so I might change one of the L which is a length of the needle into a radius of the cylinder. 

Notes on drag coefficient research
The drag force that we are looking for should be proportional to first order in velocity of the object. This should be reasonable as for small objects the reynold number is small, less than 1 and in this limit the stoke's law is a good description. Stoke's law only applies to spherical objects. 
Useful links:

1.https://www.quora.com/Is-Stokes-law-F-6%CF%80r%C2%A3v-applicable-for-cylinder-shapes-or-any-other-shape - states the general drag force for any object
2.https://phys.libretexts.org/Bookshelves/Classical_Mechanics/Book%3A_Classical_Mechanics_%28Dourmashkin%29/08%3A_Applications_of_Newtons_Second_Law/8.06%3A_Drag_Forces_in_Fluids - stokes law, drag forces and viscosity coefficients
3.https://scihubtw.tw/10.1016/0375-9601(79)90040-9 - paper about force on a rotating cylinder with holes
4.https://mae.ufl.edu/~uhk/STOKES-DRAG-FORMULA.pdf - derivation of stokes law
5.https://phys.libretexts.org/Bookshelves/Classical_Mechanics/Supplemental_Modules_%28Classical_Mechanics%29/Fluids/1.7%3A_Stokes%E2%80%99_Law - stokes law dimensional analysis


## Choosing time steps 
I might want to investigate a bit more on what the appropriate timesteps should be, Ian in his notes says that the time steop dt is required to be larger than the time at which the random changes happen but that the later goes to zero so there is almost no limit on the dt which can be made small. Let's start of with dt in microseconds. Found an article online that showed that on average molecules of N2 at room temperature and pressure colide with frequency of $10^9Hz$. https://www.tec-science.com/thermodynamics/kinetic-theory-of-gases/mean-free-path-collision-frequency/

In [4]:
def t_steps(tf, N):
    '''   
    Creates an array of N+1 evenly spaced t values from 0 to tf, this corresponds to N time steps,
    the second stop argument is not included so in order to include the tf one step tf/N was added.
    
    Parameters
    tf: final time, time period of solving the equation
    N:  number of time steps
    Rerturns
    -----
    N+1 values of t values between (0,tf) time interval
    '''
    return np.arange(0, tf + tf/N, tf/N)

t_f = 1e-4  # final time
N = int(1e5)
h = t_f/N
t_points = t_steps(t_f, N)

## Defining useful constants
which will speed up the caluclations when evaluating the differential equations. Just collecting all the constants so that they dont have to be calculated over and over again.

In [5]:
# system magnet
S1 = u_0*mu_s/(4*np.pi*d**(3)*I_s)
S2 = B_0*mu_s/I_s
S3 = gam_s/I_s
S4 = b_s/I_s
# demon magnet
D1 = u_0*mu_s/(4*np.pi*d**(3)*I_d)
D2 = gam_d/I_d
D3 = b_d/I_d

It is correct that the fluctuating force is bigger on system but it makes the demon move faster because the moment of inertia overweights the strength b of noise term.

In [6]:
print("ratio of the strength of the fluctuating forces\n", b_s/b_d)

ratio of the strength of the fluctuating forces
 2.8574404296988045


In [7]:
alpha = mu_d_max*5/t_f   # rate at which the magnetization is increasing

def mu_d(t):
    """calculates the magnetic moment of the demon paramagnet"""
    if (t <= t_f/5):
        return alpha*t
    else:
        return mu_d_max # m = Md*V

In [8]:
def langevin(r,t_points):
    ''' Calculates the right hand sight of the diff eqs
    r is a vector containing: r = [[theta_s, w_s],
                                  [theta_d, w_d]]
    '''
    # initializing empty arrays that will store the resulting trajectories
    theta_s = np.empty(t_points.shape, dtype = 'float64')
    theta_d = np.empty(t_points.shape, dtype = 'float64')
    w_s = np.empty(t_points.shape, dtype = 'float64')
    w_d = np.empty(t_points.shape, dtype = 'float64')
    
    # time step
    h = t_points[-1]/len(t_points)
    
    # storing the initial values
    theta_s[0] = r[0,0]
    theta_d[0] = r[1,0]
    w_s[0] = r[0,1]
    w_d[0] = r[1,1]
    
    # solving the dif-eq
    for idx in range(1,len(t_points)):
        t = t_points[idx] # current time
        # abreviation for variables at t
        th_s_t =  theta_s[idx-1]
        th_d_t = theta_d[idx-1]
        w_s_t = w_s[idx-1]
        w_d_t = w_d[idx-1]
        # calculate the variables at t+dt
        theta_s[idx] = th_s_t + h*w_s_t
        theta_d[idx] = th_d_t + h*w_d_t
        w_s[idx] = (w_s_t + h*S1*mu_d(t)*(np.sin(th_s_t)*np.cos(th_d_t) + 2*np.sin(th_d_t)*np.cos(th_s_t)) -
                            h*S2*np.sin(th_s_t) -
                            h*S3*w_s_t +
                            h**(1/2)*S4*np.random.normal()  
                    )
        w_d[idx] = (w_d_t + h*D1*mu_d(t)*(np.sin(th_d_t)*np.cos(th_s_t) + 2*np.sin(th_s_t)*np.cos(th_d_t)) -
                            h*D2*w_d_t  +
                            h**(1/2)*D3*np.random.normal()  
                   )

    return np.array([[theta_s, w_s],[theta_d, w_d]], dtype = 'float64')

In [9]:
# initial conditions, setting initial values for magnet's angles and velocities
init = np.array([[0,0],[np.pi,0]], dtype = 'float64')
solution = langevin(init,t_points)

In [10]:
def plot_angles(sol,t_points):
    '''Plots the angle trajectories'''
    plt.figure(figsize=(10.5, 4.2))

    plt.subplot(1,2,1)
    plt.plot(t_points,sol[0,0])
    plt.grid(True)
    plt.xlabel("t (s)")
    plt.ylabel("$\\theta_s$"+" (rad)")
    plt.title("System magnet")

    plt.subplot(1,2,2)
    plt.plot(t_points,sol[1,0])
    plt.grid(True)
    plt.xlabel("t (s)")
    plt.ylabel("$\\theta_d$"+" (rad)")
    plt.title('Demon magnet')
    
plot_angles(solution,t_points)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Plotting a 3D graph of the system-demon interaction potential $U_{int}$ 
This might be usefull so I see where the minimas are. 

In [11]:
ths = np.outer(np.linspace(-np.pi, np.pi, 50), np.ones(50))
thd = ths.copy().T # transpose

U_int_plot =  mu_d_max*u_0*mu_s/(4*np.pi*d**(3)) * (np.cos(ths)*np.cos(thd) - 2*np.sin(ths)*np.sin(thd) )

fig = plt.figure()
ax = plt.axes(projection='3d')

ax.plot_surface(ths, thd, U_int_plot/E_unit,cmap='viridis', edgecolor='none')
ax.set_title('$U_{int}(\\theta_s,\\theta_d)$')
ax.set_xlabel("$\\theta_s$")
ax.set_ylabel("$\\theta_d$")
ax.set_zlabel("U (kT)")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

So there are two minima if we let both angles to vary. I really should have analyzed the potential function earlier to get a better idea of where do the magnets tend to go. The two minima are when both the mangets lie flat parallel and their magnetic moments point in the same direction. This changes my entire perception of the dynamics between the system and the demon. So they are not trying to antialign all the time because that is not the only point where the derivative of U is zero. The entire time I thought that the magnets will try to stay antialigned to each other because that was their global minimum supposedly, but it is not. This might make the exploitation period slightly more complicated because no we cannot just say that system is pointing the opposite direction with respect to the demon. Also not to forget this potential is still missing one term the $B_0$ interaction with system magnet. Let's plot that and see what we get. Now the multiplicative constants can't be neglected.

## 3D graph of the complete U

In [12]:
U_full = mu_d_max*u_0*mu_s/(4*np.pi*d**(3))*(np.cos(ths)*np.cos(thd) - 2*np.sin(ths)*np.sin(thd)) - B_0*mu_s*np.cos(ths)

fig = plt.figure()
ax = plt.axes(projection='3d')

ax.plot_surface(ths, thd, U_full/E_unit,cmap='viridis', edgecolor='none')
ax.set_title('$U_{total}(\\theta_s,\\theta_d)$')
ax.set_xlabel("$\\theta_s$")
ax.set_ylabel("$\\theta_d$")
ax.set_zlabel("U (kT)")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Okay so I ranged the B0 from around 0.001 to 0.03 to see how it affects the potential energy. When the B0 was around the 0.001 the interaction only potential function was close to being unchanged at all. The minima remained roughly at the same angles. When the B0 was cranked up al the way to 0.03 the minima were slowly pushed to the $\theta_s=0$ angle but the potential was getting deeper and steeper, harder for the particles to move away from their minimas.

In [13]:
print("Value of B0 for which the potential U turns into a minima at the point theta_s=0 and theta_d=pi\nB0 =",
      3*mu_d_max*u_0/(4*np.pi*d**3))

Value of B0 for which the potential U turns into a minima at the point theta_s=0 and theta_d=pi
B0 = 0.011780972450961723


## Defining potential energy U functions

In [14]:
@njit
def U_int(th_s,th_d, mu_d):
    '''Potential function of the demon system interaction'''
    return mu_d*u_0*mu_s/(4*np.pi*d**(3)) * (np.cos(th_s)*np.cos(th_d) - 2*np.sin(th_s)*np.sin(th_d))

@njit
def U_system(th_s):
    '''Potential of the system coil interaction'''
    return -mu_s*B_0*np.cos(th_s)

In [15]:
U_0_min = U_system(0)
U_int_min = U_int(0,-np.pi,mu_d_max) # this isnt really the minimum of that function :) but the ths=0 and thd=PI point, it would be the zero point of the combined potential energy
print("The minimum potential energy of the system and coil\nU_0   minimum:", U_0_min)
print("The minimum potential energy of the demon and system intreaction\nU_int minimum:", U_int_min)

The minimum potential energy of the system and coil
U_0   minimum: -6.283185307179586e-17
The minimum potential energy of the demon and system intreaction
U_int minimum: -1.2337005501361697e-17


## Measuring correlation between the demon and system
There is a covariance function that measures the correlation between two random variables.
$$cov(\theta_s,\theta_d)=〈\theta_s\theta_d〉-〈\theta_s〉〈\theta_d〉$$
and then there is the correlation coefficient which is a good measure of how much the variables are correlated.
$$cor(\theta_s,\theta_d)= \frac{cov(\theta_s,\theta_d)}{\sqrt{var(\theta_s)*var(\theta_d)}}$$

The correlation takes values between -1 and 1. It goes from anticorrelated for -1 to not correlated at all/independend at 0 and maximaly correlated at 1.
The arrays of angles at different times should suffice to calculate all the above quantities.
I would like to return to the correlation measure and try to get a better understanding why it gives a reliable measure of extend to which the two variables are correlated.

In [16]:
def covariance(x,y):
    '''calculates the covariance between two variables, could have used the native numpy implementation of this function, but nevermind'''
    x_avg = np.average(x)
    y_avg = np.average(y)
    xy_avg = np.average(x*y)
    return xy_avg - x_avg*y_avg

def correlation(x,y):
    '''outputs the correlation coefficient of two random variables'''
    return covariance(x,y)/np.sqrt(np.var(x)*np.var(y))

cor = correlation(solution[0,0],solution[1,0])
print("Correlation:",cor)

Correlation: -0.8054140358893986


Assuming that the correlation and covariance functions work as intended. The correlation of almost -0.8 seems very high for $dt=10^{-9}$, but I suppose that it is correct. 

Using this covariance measure as the only measure of correlation between the two random variables can give us a wrong perception of the actual correlation. As seen in one of the ML textbooks, there are cases when the two variables are correlated or connected in a way that slips the detection through the covariance measure which is supposedly linear in a way. 

Another way to improve our intuition on the magnet's strength of coupling would be to plot a scatter and to calculate the mutual information as proposed by Ford.

### Calculating the average correlation over multiple runs


In [17]:
def cor_avg(init,t_points,runs, langevin):
    '''Calculates the correlation coefficient of the demon and system angles over multiple runs and averages those results
    Parameters
    ----------
    runs: number of runs of solving the equations'''
    cors = np.empty(runs, dtype='float64') # stors the correlation coefficients for each run
    
    for run in range(runs): 
        solution = langevin(init,t_points, cpl_period = 1/100000) # fast coupling, want to measure the correlation only when the magnets are fully coupled
        cor = correlation(solution[0,0],solution[1,0])
        cors[run] = cor

    return np.average(cors)

### Scatter plot

In [ ]:
def plot_scatter(sol):
    'plots a scatter plot of the demon and system angles'
    plt.figure()

    plt.scatter(sol[0,0],sol[1,0], s=2, color = (0.2, 0.5, 0.4), alpha = 0.01)
    plt.xlabel("$\\theta_s (rad)$")
    plt.ylabel("$\\theta_d (rad)$")
    plt.title("Scatter plot")

plot_scatter(langevin(init, t_steps(1e-4, 1e5)))
plt.savefig("1.svg")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

This is somewhat informative in a way that it tells us that the magnet's angles really are linearly dependent to a certain approximation validity I guess.

## Divergence of the equations of motion
The demon's solution started diverging at about $dt=5.2*10^{-8}s$ which is very close to the theoretical value calculated below when only resistance term was cosidered. We had to go much further with the dt for the system to also diverge. It did so somewhere around the $6.5*10^{-8}s$. It could have been a bit sooner but that is not important, what matters is that the dt at which the solutions diverges is approximately around the range of the theoreticaly calculated values given below. The system was less prone to diverging because its movement is additionaly modulated by the $B_0$ field.

In [19]:
max_dt_s = 2*I_s/gam_s
max_dt_d = 2*I_d/gam_d
print("demon  dt:", max_dt_d)
print("system dt:", max_dt_s)

demon  dt: 5.173017847809882e-08
system dt: 5.622083333333331e-08


## Comparison of the demon's orientation with respect to its minimum energy angle
This will give us a rough idea of how closely the demon follows the system's magnetic orientation. This should be a good measure of it beacuse once the system magnet moves out of its equilibrium the demon's minimum energy state changes, if the demon was perfectly copying the system's motion we would always find it in this minim angle state.

In [20]:
def plot_correlation(sol, t_points):
    '''plots the demons angle trajectory in comparison to its minimum potential angle trajectory'''
    th_d_follow = np.arctan(-2*np.tan(sol[0,0])) + np.pi # had to shift this by -pi because of the range of tan and arctan values
    plt.figure()
    
    plt.plot(t_points,sol[1,0], label="demon")
    plt.plot(t_points,th_d_follow, label="system")
    plt.grid(True)
    plt.xlabel("t (s)")
    plt.ylabel("$\\theta_d$"+" (rad)")
    plt.title("Demon following system")
    plt.legend(loc="best")
    
plot_correlation(solution, t_points)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Yeah that epxlains why the correlation between the two angles was so high. The demon is doing a good job in following the system's movement but is not as quick to sudden changes of direction. The graph above really isn't really comparing the demon's and system's angles but rather the demon's angle with what the demon's angle would be if it was perfectly coupled to system and follow it instantaneously. 

# Langevin numba

In [21]:
@njit
def langevin_numba(r,  t_points, cpl_period = 1/5, dcpl_period = 0,  coupling = 'linear'):
    ''' Solves the differential equations and returns an array containing the angle trajectories with all the velocities
    Parameters
    ----------
    r: is a vector containing  the initial boundary values r = [[theta_s, w_s],
                                                                [theta_d, w_d]]
    cpl_period: a fraction of time it takes to couple the demon to the system, originaly 1/5, default is the original
    dpl_period: a fraction of time it takes to decouple the demon from system, default 0, instantaneous decoupling
    coupling: type of coupling, linear, and exponential were implemented so far
    t_points: array of t points to solve the equations for
    
    Returns
    -------
    sol: (2,2,t_points.size) array containing the solved angles and velocities
    
    '''
    # initializing empty arrays that will store the resulting trajectories
    theta_s = np.empty(t_points.shape)
    theta_d = np.empty(t_points.shape)
    w_s = np.empty(t_points.shape)
    w_d = np.empty(t_points.shape)
    
    # t final
    t_f = t_points[-1]
    h = t_f/(len(t_points)-1)
    
    # storing the initial values
    theta_s[0] = r[0,0]
    theta_d[0] = r[1,0]
    w_s[0] = r[0,1]
    w_d[0] = r[1,1]

    # solving the dif-eq
    for idx in range(1,len(t_points)):
        t = t_points[idx]
        # abreviation for variables at t
        th_s_t =  theta_s[idx-1]
        th_d_t = theta_d[idx-1]
        w_s_t = w_s[idx-1]
        w_d_t = w_d[idx-1]
        # calculate the variables at t+dt
        theta_s[idx] = th_s_t + h*w_s_t
        theta_d[idx] = th_d_t + h*w_d_t
        w_s[idx] = (w_s_t + h*S1*mu_d_numba(t,t_f, cpl_period, dcpl_period = dcpl_period, coupling = coupling)*(np.sin(th_s_t)*np.cos(th_d_t) + 2*np.sin(th_d_t)*np.cos(th_s_t)) -
                            h*S2*np.sin(th_s_t) -
                            h*S3*w_s_t +
                            h**(1/2)*S4*np.random.normal()  
                    )
        w_d[idx] = (w_d_t + h*D1*mu_d_numba(t,t_f, cpl_period, dcpl_period = dcpl_period,  coupling = coupling)*(np.sin(th_d_t)*np.cos(th_s_t) + 2*np.sin(th_s_t)*np.cos(th_d_t)) -
                            h*D2*w_d_t  +
                            h**(1/2)*D3*np.random.normal()  
                   )
        
    sol = np.empty((2,2, t_points.shape[0]))
    sol[0,0] = theta_s
    sol[0,1] = w_s
    sol[1,0] = theta_d
    sol[1,1] = w_d
    
    return sol

@njit
def mu_d_numba(t, t_f, cpl_period, coupling = 'linear', dcpl_period = 0):
    """calculates the magnetic moment of the demon paramagnet
    Parameters
    ----------
    t_f: final time of the time interval period
    coupling: type of coupling chosen, LINEAR, EXPONENTIAL
    """
    
    # turning on coupling
    if (t <= t_f*cpl_period):
        # linear coupling, default
        if (coupling == 'linear'):
            alpha = mu_d_max / (t_f*cpl_period)    
            return alpha*t
        
        # exponential coupling
        elif (coupling == 'exp'):
            gam = 10 / (cpl_period*t_f)
            #if t > t_p[int(N*cpl_period/10 - 5)] and t < t_p[int(N*cpl_period/10)]:
                #print(mu_d_max * (1 - np.exp(-gam*t)))
            return mu_d_max * (1 - np.exp(-gam*t))
    
    # decoupling
    elif (t > t_f*(1-dcpl_period)):
        # linear coupling 
        if (coupling == 'linear'):
            alpha = mu_d_max / (t_f*dcpl_period) 
            return mu_d_max - alpha*(t - t_f*(1-dcpl_period))
        # exponential decoupling
        elif (coupling == 'exp'):
            gam = 10 / (t_f*dcpl_period)
            return mu_d_max * np.exp(-gam*(t - t_f*(1-dcpl_period)))
            
            
    # max coupling  
    else:
        return mu_d_max # m = Md*V

In [22]:
t_p = t_steps(1e-5, 1e6)
r1 = np.array([[0,0],[0,0]])
sol_exp = langevin_numba(init, t_p,
                         cpl_period = 1/5,
                         dcpl_period = 1/5,
                         coupling = 'linear')

In [23]:
#plot_angles(sol_exp, t_p)
plot_correlation(sol_exp, t_p)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Benchmarking numba version of the langevin function
Unbeliveable that we the function gets sped up by 200 times and the for loop seems to stay untouched, it still computes it it in series. I mean this still seems to big of an improvement for a nonparallelized for loop. It is tricky to validate the numba implementation of the function because of the np.random which causes each solution of the equations to be unique. One thing that I was worried about was that the for loop inside the @njit decorator gets subdivided into multiple parts and evaluated simultaneously in parallel which just cant be done when solving a time evolving equations of motion. If this would be happening that I suspected that there would be jumps in the solutions since for the first elemnt in each section it would have to take the previous element which was not yet computed and set by np.empty to very small value. This cant be seen in the graphs. Moreover I put a print statements inside that executed every 10 iterations and they executed in sequence. Well I might try to look into this tomorrow. One more thing that sort of points in the direction that the function works as intended is the fact that it produces same correlations as normal implementation over the entire range of dts that were tested. There was still something odd happening when evaluating the functions (have to make sure that all variables got updated). 

When parallel se to True, it outputs an error message that there was nothing that could be parallelized. The function still runs and takes the same amount of time to execute. Additionaly, I ran the langevin numba function again with print statements inside but now printing on each iteration and it was printing the values in sequence again. I tried this logic on a simple function that was parallelized with parallel = True and used prange and indeed the print statements did not execute in sequence. Though the prange had to be included otherwise the for loop did not get parallelized. Alright this should settle down my worries of the langevin not executing in series.

And final remark, python for loops are unbeliveably slow if numba can speed them up by a factor of 250 without parallelizing it.

In [24]:
%timeit langevin_numba(init, t_points)
%timeit langevin(init, t_points)

12.4 ms ± 370 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
3.77 s ± 1.12 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


By getting rid of the if statement and looping over a range of indecis instead of enumerate the execution time for classical langevin equation was brougt down from 4 to 2.7s, the numba implementationdid not change almsot at all

## Average correlation over multiple runs

In [25]:
t_points4 = t_steps(10**(-5), int(1e6))
cor_avg_many = cor_avg(init, t_points4,50, langevin_numba)
print('The average correlation coefficient value:',cor_avg_many)

The average correlation coefficient value: -0.6944632253757639


## Equilibrium miscrostate probability of demon and system
when the coupling is fully turned on.

In [101]:
t_long = t_steps(10**(-3), int(1e7))
sol_pdf = langevin_numba(init, t_long, cpl_period = 1/100)

In [102]:
#plot_angles(sol_pdf, t_long)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [103]:
plt.figure(figsize=(10.5, 4.2))

num_bins = 40
plt.subplot(1,2,1)
n, bins, patches = plt.hist(sol_pdf[1,0][-int(9e6):], num_bins)
plt.xlabel('$\\theta_d$')
plt.ylabel('counts')
plt.title('$\\theta_d$ equilibrium distribution')
plt.subplot(1,2,2)
n, bins, patches = plt.hist(sol_pdf[0,0][-int(9e6):], num_bins)
plt.xlabel('$\\theta_s$')
plt.ylabel('counts')
plt.title('$\\theta_s$ equilibrium distribution')
pdf_s_std = np.std(sol_pdf[0,0][-int(9e6):])
pdf_d_std = np.std(sol_pdf[1,0][-int(9e6):])
print("Demon's pdf\nmean: ", np.mean(sol_pdf[1,0][-int(9e6):]), "\nstd:  " , pdf_d_std)
print("System's pdf\nmean: ", np.mean(sol_pdf[0,0][-int(9e6):]), "\nstd:  " ,pdf_s_std)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Demon's pdf
mean:  3.1402093846638888 
std:   0.03127744727933691
System's pdf
mean:  0.0005601544354923203 
std:   0.012804431800403252


# Mean rotatinal kinetic energy
compared to $kT/2$ which it should equal to according to equipartition theorem and also by construction of langevin equations because it the fluctuating and resistive terms were set such that they satisfy this statistical property of particles in equilibrium with a heat bath/rest of the fluid.

In [29]:
@njit(['float64(float64[:], float64)'])
def avg_KE(w,I):
    '''
    Parameters
    ----------
    w: an array of angular velocities, usually solved over 1 run
    I: the moment of inerta
    Funny that numba does not yet know yet how to convert np.average to machine code but has no problem with np.mean
    '''
    E_k = 1/2*I*w**2
    return np.mean(E_k)

sol = langevin_numba(init, t_points, cpl_period = 1/5)
KE = avg_KE(sol[0,1],I_s)
KE_D = avg_KE(sol[1,1],I_d)
print('1 run \nAverage kinetic energy over coupling and stationary period (mu_d does not change)')
print('<Ek> of the system:',KE*2/E_unit)
print('<Ek> of the demon :',KE_D*2/E_unit)
print('kT/2              :',1)
print('Average kinetic energy over stationary period')
print('<Ek> of the system:',avg_KE(sol[0,1][int(N/5):],I_s)*2/E_unit)
print('<Ek> of the demon :',avg_KE(sol[1,1][int(N/5):],I_d)*2/E_unit)

1 run 
Average kinetic energy over coupling and stationary period (mu_d does not change)
<Ek> of the system: 1.0545104145042066
<Ek> of the demon : 1.051421089545789
kT/2              : 1
Average kinetic energy over stationary period
<Ek> of the system: 1.070107577513761
<Ek> of the demon : 1.0398045578638884


In [30]:
@njit(parallel = True)
def avg_KE_stats(n, runs, t_points, N, langevin, cpl_period = 1/5, coupling = 'linear'):
    ''' average of average of averages,
    Outputs n averaged kinetic energies each averaged over #runs of runs of the solutions
    Calculates the average kinetic energies over the coupling period and over the stationary period at max coupling separately.
    Parameters
    ----------
    runs: is the number of runs per each cycle
    n: number of cycles, in each cycle #runs of runs are executed  
    '''
    # stores the average kinetic energies of each cycle (coupling period included)
    KE_cycle_couple_s = np.empty(n)
    KE_cycle_couple_d = np.empty(n)
    
    # stationary period
    KE_cycle_stat_s = np.empty(n)
    KE_cycle_stat_d = np.empty(n)
    
    # potential energy
    U_cycle = np.empty(n)
    
    for cycle in prange(n):
        # stores the average kinetic energies of each run/solution
        KE_run_couple_s = np.empty(runs)
        KE_run_couple_d = np.empty(runs)
        
        # stationary period
        KE_run_stat_s = np.empty(runs)
        KE_run_stat_d = np.empty(runs)
        
        #U_run = np.empty(runs)
        
        for run in prange(runs):
            sol = langevin(init, t_points, cpl_period = cpl_period, coupling = coupling)
            KE_run_couple_s[run] = avg_KE(sol[0,1][:int(N/5)], I_s)
            KE_run_couple_d[run] = avg_KE(sol[1,1][:int(N/5)], I_d)
            
            KE_run_stat_s[run] = avg_KE(sol[0,1][int(N/5):], I_s)
            KE_run_stat_d[run] = avg_KE(sol[1,1][int(N/5):], I_d)
            #U_run[run] = 
            
        # after all the runs of this cycle are computed we can take the next average :)    
        KE_cycle_couple_s[cycle] = np.mean(KE_run_couple_s)
        KE_cycle_couple_d[cycle] = np.mean(KE_run_couple_d)
        KE_cycle_stat_s[cycle] = np.mean(KE_run_stat_s)
        KE_cycle_stat_d[cycle] = np.mean(KE_run_stat_d)
        
    return KE_cycle_couple_s, KE_cycle_couple_d, KE_cycle_stat_s, KE_cycle_stat_d        

In [31]:
# calculating the statitics of average kinetic energies over set of multiple runs each called a cycle 100x100 solutions of langevin
KE_100_couple_s, KE_100_couple_d, KE_100_stat_s, KE_100_stat_d = avg_KE_stats(10,100, t_points, N, langevin_numba)

In [32]:
# Energies expressed in units of kT/2
print('100x100 runs\nAverage and std of 100 cycles of averaged kinetic energies over 100 runs of langevin equations \nSystem')
print('max coupling    : {0:.5f} +/- {1:.5f}'.format( np.average(KE_100_stat_s)*2/E_unit, np.std(KE_100_stat_s)*2/E_unit) )
print('during coupling : {0:.5f} +/- {1:.5f}'.format( np.average(KE_100_couple_s)*2/E_unit, np.std(KE_100_couple_s)*2/E_unit),  '\nDemon') 

print('max coupling    : {0:.5f} +/- {1:.5f}'.format( np.average(KE_100_stat_d)*2/E_unit, np.std(KE_100_stat_d)*2/E_unit) )
print('during coupling : {0:.5f} +/- {1:.5f}'.format( np.average(KE_100_couple_d)*2/E_unit, np.std(KE_100_couple_d)*2/E_unit) )

100x100 runs
Average and std of 100 cycles of averaged kinetic energies over 100 runs of langevin equations 
System
max coupling    : 1.02072 +/- 0.00229
during coupling : 1.02014 +/- 0.00336 
Demon
max coupling    : 1.02345 +/- 0.00276
during coupling : 1.02191 +/- 0.00416


Above I measured some statistical properties of the rotational kinetic energies of the system and demon during two separate periods, first while the coupling is turned on maximum and does not change and for second period which is 'during coupling' period that takes place while coupling is beng turned on from 0 to $kT/2$.

Clearly the mean average kinetic energies over the two period are very close and agree within the 1 standard deviation, this gives us a clue that the coupling process does not affect the average kinetic energies as much and it definitely does not explain the 2% discrepency with the equipartition theorem. Still interesting to notice that the coupling period gets closer to the equipartition theorem (I ran this multiple times and it seemed to be the case even thought the std is quite high).

Let's try to decrease the time step $dt$ and and see if the average kinetic energy gets closer to what it is supposed to be.

In [33]:
# Calculating the average kinetic energies again but now with shorter dt 
N1 = int(1e5)
t_f1 = 10**(-5)
t_points_1 = t_steps(t_f1,N1)
KE_100_couple_s1, KE_100_couple_d1, KE_100_stat_s1, KE_100_stat_d1 = avg_KE_stats(10,100, t_points_1, N1, langevin_numba)

In [34]:
# Energies expressed in units of kT/2
print('100x100 runs\nAverage and std of 100 cycles of averaged kinetic energies over 100 runs of langevin equations \nSystem')
print('max coupling    : {0:.5f} +/- {1:.5f}'.format( np.average(KE_100_stat_s1)*2/E_unit, np.std(KE_100_stat_s1)*2/E_unit) )
print('during coupling : {0:.5f} +/- {1:.5f}'.format( np.average(KE_100_couple_s1)*2/E_unit, np.std(KE_100_couple_s1)*2/E_unit),  '\nDemon') 

print('max coupling    : {0:.5f} +/- {1:.5f}'.format( np.average(KE_100_stat_d1)*2/E_unit, np.std(KE_100_stat_d1)*2/E_unit) )
print('during coupling : {0:.5f} +/- {1:.5f}'.format( np.average(KE_100_couple_d1)*2/E_unit, np.std(KE_100_couple_d1)*2/E_unit) )

100x100 runs
Average and std of 100 cycles of averaged kinetic energies over 100 runs of langevin equations 
System
max coupling    : 0.99769 +/- 0.00614
during coupling : 0.99237 +/- 0.01280 
Demon
max coupling    : 1.00280 +/- 0.00629
during coupling : 1.00714 +/- 0.01820


The mean kinetic energies of both system and demon agree up to 2 significant digits with the $kT/2$

The dt was decreased by a factor of $10^{-3}$ and there is no clear improvement in terms of the kinetice energy approaching what is predicted by equipartition theorem. The coupling a fully coupled period do seem to exhibit a slight difference but that is still negligible. - This was due to a bug in the code.

HYPOTHESIS: Okay so we assumed that the average kinetic energy of the magnet will be equal to $kT/2$ as it was stated by equipartition theorem, but that is not what the theorem says. It states that when ... maybe not

Okay so the 2 percent discrepency between still has not been resolved and we came up with several guesses of what might the cause of it. 

1. The np.random.normal function might be outputing truly random numbers


I checked that and it pretty much does, the std and mean both matched for and average and std over $10^8$ of these randomly drawn numbers the average and standard deviation agree up to an error of 5 significant figures, the mean was with an error of $10^{-5}$. Same results obtained when the random function was called from within a numba decorator.

2. There is mistake/typo in the code or in the derivation of the equations of motion themself.

3. The method for numericaly solving the equations is not accurate enough.

The kinetic energies consistenly are higher than what they should be, the demon always overshoots more than the system magnet which might suggest that the more the magnets are allowed to move the more their kinetic energy exceeds the equipartition theorem.

### Mean kinetic energy of free magnets

In [35]:
@jit
def langevin_free(r,t_points, cpl_period = 0, coupling = 'None'):
    ''' Calculates the right hand sight of diff eqs for free to move magnets, no potential interactions
    r is a vector containing r = [[theta_s, w_s],
                                  [theta_d, w_d]]
    I had to make this function accept cpl period and coupling so we do not get error when this function gets called inside the avg_KE_stat,
    '''
    # initializing empty arrays that will store the resulting trajectories
    theta_s = np.empty(t_points.shape)
    theta_d = np.empty(t_points.shape)
    w_s = np.empty(t_points.shape)
    w_d = np.empty(t_points.shape)
    
    # t final
    t_f = t_points[-1]
    h = t_f/ (len(t_points) - 1)
    # storing the initial values
    theta_s[0] = r[0,0]
    theta_d[0] = r[1,0]
    w_s[0] = r[0,1]
    w_d[0] = r[1,1]

    # solving the dif-eq
    for idx in range(1,len(t_points)): 
        # abreviation for variables at t
        th_s_t =  theta_s[idx-1]
        th_d_t = theta_d[idx-1]
        w_s_t = w_s[idx-1]
        w_d_t = w_d[idx-1]
        # calculate the variables at t+dt
        theta_s[idx] = th_s_t + h*w_s_t
        theta_d[idx] = th_d_t + h*w_d_t
        w_s[idx] = (w_s_t - h*S3*w_s_t  + h**(1/2)*S4*np.random.normal() )
        w_d[idx] = (w_d_t - h*D2*w_d_t  + h**(1/2)*D3*np.random.normal() )
        
    sol = np.empty((2,2, t_points.shape[0]))
    sol[0,0] = theta_s
    sol[0,1] = w_s
    sol[1,0] = theta_d
    sol[1,1] = w_d
    
    return sol

In [36]:
sol_free = langevin_free(init, t_points)
plot_angles(sol_free,t_points)
print('correlation:',correlation(sol_free[0,0],sol_free[1,0]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

correlation: -0.49046645489412793


In [37]:
KE_free_s1, KE_free_d1, KE_free_s2, KE_free_d2 = avg_KE_stats(10,100, t_points_1, N1, langevin_free)

In [38]:
KE_free_s_avg = np.average( (KE_free_s1 + 4*KE_free_s2)/(5*E_unit) )*2
KE_free_d_avg = np.average( (KE_free_d1 + 4*KE_free_d2)/(5*E_unit) )*2
KE_free_s_std = np.std( (KE_free_s1 + 4*KE_free_s2)/(5*E_unit) )*2
KE_free_d_std = np.std( (KE_free_d1 + 4*KE_free_d2)/(5*E_unit) )*2

print('system\n<KE_cycle>:{0:.5f} +/- {1:.5f}'.format(KE_free_s_avg ,KE_free_s_std ) )
print('demon\n<KE_cycle>:{0:.5f} +/- {1:.5f}'.format(KE_free_d_avg, KE_free_d_std ) )

system
<KE_cycle>:0.99992 +/- 0.00735
demon
<KE_cycle>:0.99785 +/- 0.00914


### KE over a single run with high number of steps N

In [39]:
N2 = int(1e7)
t_f2 = 10**(-3)
t_points2 = t_steps(t_f2, N2)

sol2 = langevin_free(init,t_points2)
print('system\n<KE_cycle>:{0:.5f}'.format( (avg_KE(sol2[0,1],I_s))*2/E_unit  ) )
print('demon\n<KE_cycle>:{0:.5f} '.format( (avg_KE(sol2[1,1],I_d))*2/E_unit  ) )

system
<KE_cycle>:0.99703
demon
<KE_cycle>:1.00196 


## Average potential energy of the magnets

In [40]:
# function that calculates the average potential energy per run
@njit(parallel = True)
def U_avg(sol,t_points,cpl_period = 1/5):
    ''' It is easiest to measure it over the max coupling period
    '''
    N = t_points.shape[0]
    u_len = int(N*(1-cpl_period)) # length of the max coupling period
    i_shift = int(N*cpl_period)   # start index of the max coupling period
    
    u_sys = np.zeros(u_len)
    u_int = np.zeros(u_len)
    
    tf = t_points[-1]
    for i in prange(u_len):
        u_sys[i] = U_system(sol[0,0][i+i_shift])
        u_int[i] = U_int(sol[0,0][i+i_shift], sol[1,0][i+i_shift], mu_d_numba(t_points[i+i_shift], tf, cpl_period = cpl_period))
        
    u_tot = u_sys + u_int

    return np.mean(u_sys), np.mean(u_int), np.mean(u_tot)

In [41]:
umean = U_avg(sol,t_points)[-1]
print( (umean - U_0_min - U_int_min)/E_unit) 

1.049423378965749


In [42]:
@njit(parallel=True)
def U_avg_stats(n, runs, langevin):
    '''Calculates the average potential energy over n cycles of #runs of runs, it calculates the avg potential energy only over the fully coupled period
    from N/5 to N steps
    !!!! There is something odd happening when the outer loop gets parallelized with prange(), might want to investigate why we were getting 10^50 as avg values
    of the potential energy, it makes me worried because we used an exactly same code structure for the avg KE but fortunately it did not break. Might want to
    get a better understand what is this cause by. !!!!
    '''
    U_tot_cycles = np.zeros(n)
    U_sys_cycles = np.zeros(n)
    U_int_cycles = np.zeros(n)
    
    for cycle in range(n):
        U_tot_runs = np.zeros(runs)
        U_sys_runs = np.zeros(runs)
        U_int_runs = np.zeros(runs)
        
        for run in prange(runs):
            sol = langevin(init, t_points_1)
            U_sys_runs[run], U_int_runs[run], U_tot_runs[run] = U_avg(sol, t_points_1)
            
        U_tot_cycles[cycle] = np.mean(U_tot_runs)
        U_int_cycles[cycle] = np.mean(U_int_runs)
        U_sys_cycles[cycle] = np.mean(U_sys_runs)
    
    return U_sys_cycles, U_int_cycles, U_tot_cycles 

In [43]:
uavg = U_avg_stats(10, 100, langevin_numba)

print("<U_int>: {0:.5f} +/- {1:.5f}".format( np.average(uavg[1]- U_int_min)/E_unit , np.std(uavg[1]- U_0_min)/E_unit ) )
print("<U_sys>: {0:.5f} +/- {1:.5f}".format( np.average(uavg[0]- U_0_min)/E_unit , np.std(uavg[0] - U_int_min)/E_unit ) )
print("<U_tot>: {0:.5f} +/- {1:.5f}".format( np.average(uavg[2]- U_0_min - U_int_min)/E_unit , np.std(uavg[2]- U_0_min - U_int_min)/E_unit ) )

<U_int>: -0.20267 +/- 0.03878
<U_sys>: 1.19468 +/- 0.06534
<U_tot>: 0.99201 +/- 0.03128


This seems too good to be true. Very interesting to see that the average potential energy is very close to kT, each potential function contributes kT/2, one coming from the magnet magnet interaction and the other from the system interacting with the coil. Ian suggested that the sines and cosines in the potential fucntion can be expanded around its minima and approximated to second order which would give us a harmonic potential which would translate to kT/2 per quadratic term contribution to total average energy according to equipartition theorem. Well, computationaly we get close to it but analyticaly I was not able to derive such a result not even with tons of approximations. Would be interesting to see how much this changes as we deacrease the strength of the potentials and the magnets explore wider range of angles. 

The average of U_system which is the potential energy between system and coil is a reflection of a maximum energy that could be extracted on account of a perfect measurement of the state of the system.

In [44]:
%timeit U_avg(sol,t_points)
%timeit avg_KE(sol[0,1],I_s)

2.46 ms ± 329 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
236 µs ± 34.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


# Energy cost of coupling 
For now we can implement a fucntion that takes in all the angles and calculates the energy cost, in future I might put this inside the langevin function or create a class and combine all this together and create a method/functions on that class.

So if we abruptly set the demon's magnetic moment to zero after the coupling period the mean energy cost seemed like to be negative indicating tat more enrgy is flowing out of the system than into the system. 

In [45]:
@njit()
def work_couple_numba(solution, t_points, cpl_period = 1/5, coupling = 'linear'):
    '''Parallelized version of the work function, important not to forget that this will produce slightly different
    results compared to a serial evaluation of the work, because of float rounding. 
    I also made it include the N step because at Nth time step the coupling is not on its max value yet.
    '''
    N = len(t_points) - 1 # not a length of the tpoints but number of time intervals
    t_f = t_points[-1]
    dt = t_f/ N 
    
    th_s = solution[0,0][:int(N*cpl_period)]
    th_d = solution[1,0][:int(N*cpl_period)]

    C = u_0*mu_s/(4*np.pi*d**(3))   # collection of constants
    
    W = 0  # total work done     

    # du/dt
    # linear
    if (coupling == 'linear'):
        alpha =  mu_d_max / (t_f*cpl_period)
        dudt = np.full(th_s.size, alpha) # just creating an array of the same dudt values, this implementation needs to be changed

    # exponential
    elif (coupling == 'exp'):
        gam = 10 / (t_f*cpl_period)
        exp_arg = -gam*t_points[:int(N*cpl_period)]
        dudt = mu_d_max * gam * np.exp(exp_arg)
        
    # evaluate the work
    for i in prange(th_s.size):
        W +=  C*( np.cos(th_s[i])*np.cos(th_d[i]) - 2*np.sin(th_s[i])*np.sin(th_d[i])) * dudt[i] * dt

    return W

cp = 1/5
w_couple =  work_couple_numba(langevin_numba(init, t_points_1, cpl_period = cp), t_points, cpl_period = cp)   
print(w_couple)

-1.2332144220285172e-17


In [46]:
sol2 = langevin_numba(init, t_points_1, cpl_period = 1/5, coupling = 'exp')
print('work:',work_couple_numba(sol2, t_points_1, cpl_period = 1/5, coupling = 'exp'))

work: -1.2339524888500346e-17


This again something completely unexpected, if we make the coupling period shorter the energy we gain from coupling increases which is exact opposite of what we would expect. The more abrupt coupling the more wasteful and heat disipative it should be but instead we see a gain in energy. It makes a sense from the point of view that when we are making the coupling period smaller and smaller we are approching the limit where we the coupling period is 0 and the energy gained is maximized - the difference between the final and initial potential energies. But otherwise I have no clue how to explain this.

In [47]:
def work_couple(solution, N ,t_f):
    '''I am not using this function anymore, just keeping it here for reference, need to be careful about the h that does not change with t_points'''
    alpha = mu_d_max*5/t_f
    
    th_s = solution[0,0][:int(N/5)]
    th_d = solution[1,0][:int(N/5)]
    
    C = u_0*mu_s/(4*np.pi*d**(3))   # collection of constants
    
    W = 0    # total work done 
    for i in range(th_s.size):
        W +=  C*( np.cos(th_s[i])*np.cos(th_d[i]) - 2*np.sin(th_s[i])*np.sin(th_d[i])) * alpha * h   
    return W

Okay this is unexpected, so I calculated the energy it takes to couple the system and magnet. Since they are already in their lowest energy, by turning on the magnetic moment of the demon we are decreasing the potential energy and the work that it takes to couple the demon with a system is therefore negative which just appears so odd at first sight. Now we will have to give the system energy to decouple it from the demon and that should be higher than the energy that we initially obtained from the system/heat bath. I dont know yet what to think about this. If the process of consuming a recovering energy from the enviroment is reversed, does it make sense to postulate that energy lost to the enviroment is quasistatic?

### Benchmarking work function performance

In [48]:
%timeit work_couple(solution, N ,t_f)
%timeit work_couple_numba(solution, t_points)

186 ms ± 29.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
703 µs ± 43.9 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


The numba version evaluates the function around 500 times faster which is a decent speed up. Well it used to be, but now when the fucntion is more complicated it is slower. The if branch slows it down by almsot a factor of 2.

# Mean energy cost of measurement
Calculating the mean energy cost of coupling the demon with the system over multiple runs and thus obtaining a statistical average of this quantity. The initial positions of both magnets are always reset.


I discovered that the interval over which the work was calculated was slightly of by 1 time step, so we did get an extra negative contribution from that time step during the coupling process which resulted in average energy cost being negative, still not sure why are we getting much.... 

In [49]:
@njit()
def E_cost(n, t_points, cpl_period = 1/5, dcpl_period = 0, coupling = 'linear', pdf = 'previous'):
    '''returns energy costs of coupling and decoupling the demon of n runs
    Parameters
    ----------
    pdf: initializes the magnets positions an velocities
        previous = sets it to values it had at the end of previous run
        eq = randomly initializes the angles and velocities with boltzmans equilibrium distribution
        zero = sets the velocities to zero, system's angle to zero and demon's angle to pi
    '''
    w_costs = np.empty(n)
    w_couples = np.empty(n)
    w_decouples = np.empty(n)
    
    init_r = init  # initializing 
    for run in range(n):
        sol = langevin_numba(init_r, t_points, cpl_period=cpl_period, dcpl_period = dcpl_period, coupling = coupling)
        
        w_couple   = work_couple_numba(sol, t_points, cpl_period = cpl_period, coupling = coupling)   #  - energy change during coupling 
        w_decouple = work_decouple(sol, t_points, dcpl_period = dcpl_period, coupling = coupling)     #  + energy change during coupling
        
        w_couples[run] = w_couple
        w_decouples[run] = w_decouple
        w_costs[run] = w_couple + w_decouple
        
        #Specifying the initial angles and velcoities for the next run
        # using the angles and velocites from previous run (instantaneous decoupling and extraction of energy)
        if (pdf == 'previous'):
            init_r = sol[:,:,-1]
            
        # angles randomized according to boltzman equilibrium distribution and the velocities set to zero, np.random.choice not recognized by numba
        elif (pdf == 'eq'):
            init_r = np.zeros((2,2))
            th_d = np.random.random()*np.pi*2 - np.pi
            th_s = np.random.normal()*pdf_s_std
            init_r[0,0], init_r[1,0] = th_s, th_d
            
        # all zeros, zero velocities and angles at their respective minima
        elif (pdf == 'zero'):
            init_r = np.zeros((2,2))
            init_r[1,0] = np.pi

    return w_couples/E_unit, w_decouples/E_unit, w_costs/E_unit

In [50]:
@njit()
def work_decouple(sol, t_points, dcpl_period = 0, coupling = 'linear'):
    ''' calculates the work done during the decoupling process
    the decoupling process starts at time step N(1-dcpl_period) + 1 even though the coupling is still 100% on, this is the first t point where the
    slope of demon's magnetic moment starts decreasing and we have to include this step.
    and we have to end at time step N because at N+1 the coupling should effectively come to zero. There was something weird happening with the indexing
    of the sol and points arrays, so it was including the N+1 element inside the indexed arrays and that made the problem assymetric, 20001 decoupling points and 
    20000 coupling points, also by including the last point an additional contribution was made towards the + decoupling work
    '''
    N = int(t_points.size - 1) # Number of time intervals
    t_f = t_points[-1]
    dt = t_f/ N 
    
    C = u_0*mu_s/(4*np.pi*d**(3))   # collection of constants
    W = 0

    # instantaneous decoupling 
    if (dcpl_period == 0):
        th_s, th_d = sol[0,0][-1], sol[1,0][-1] # final values of th_d and th_s
        return 0 - U_int(th_s, th_d, mu_d_max) # + energy lost to decouple 
    
    # continuous decoupling 
    else:
        th_d = sol[1,0][int(N*(1-dcpl_period)) : N]   # fucked up indexing :N+1 still included the N+1 element
        th_s = sol[0,0][int(N*(1-dcpl_period)) : N]   # last element is not included (N+1)
        
        # linear coupling
        if (coupling == 'linear'):
            alpha = mu_d_max / (t_f*dcpl_period)
            dudt = np.full(th_s.size, - alpha ) # just creating an array of the same dudt values, this implementation needs to be changed

        # exponential
        elif (coupling == 'exp'):
            gam = 10 / (t_f*dcpl_period)
            exp_arg = - gam * ( t_points[int(N*(1-dcpl_period)) : N] - t_f*(1-dcpl_period) )
            dudt = - mu_d_max * gam * np.exp(exp_arg)

        for i in prange(th_s.size):
            W +=  C*( np.cos(th_s[i])*np.cos(th_d[i]) - 2*np.sin(th_s[i])*np.sin(th_d[i])) * dudt[i] * dt
            
    return W
        

In [51]:
cpl = 1/5
dcpl = 1/5
solt = langevin_numba(init, t_points_1, cpl_period = cpl, dcpl_period = dcpl, coupling = 'linear')
print('WORK couple  :', work_couple_numba(solt, t_points_1, cpl_period = cpl, coupling = 'linear'))
print('WORK decouple:', work_decouple(solt, t_points_1, dcpl_period = dcpl, coupling = 'linear'))
print('U decouple:', -U_int(solt[0,0][-1], solt[1,0][-1], mu_d_max))

WORK couple  : -1.2334007125911408e-17
WORK decouple: 1.2335200066725644e-17
U decouple: 1.233492065737014e-17


I ran the E_cost function below with couplig and decoupling period 1/20, N was 10^7 and tf = 10^(-5) which makes the dt 10-12, for such a short period of coupling and decoupling the average energy cost of measurement was found to be 0.8 and 0.6 which closely relates to what we found for t_points4 but here the accuracy was 10 times higher with 500 000 time steps during both coupling and decoupling period which is a lot. If we ran this algorithm again with the same number of steps and tf then but for now a slightly lareger coupling and decoupling period of 1/5, if it produces the same mean energy cost of measurement as what we got for t_points4 with worse accuracy then it would suggest that the t_points4 is accurate enough and the unexpected trends cannot be explained by insufficient accuracy of our numerical methods. And yes this is what happend, morevover for cpl and dcpl periods of 1/5 we obtained similar results for t_points4 and t_points5, around 0.08 which I was also able to obtain with t_points_1 which is 100 times less accurate than t_points5, so yeah the method seems to be accurate enough for these t_points4 timesteps, what else is going on then?

In [52]:
# I want to run this with 10000 runs which will take about 1 and half hours, will do that while working out.
t_points5 = t_steps(10**(-5), int(10**7))
N3 = int(1e5)
t_f3 = 10**(-5)
t_points3 = t_steps(t_f3, N3)
t_points4 = t_steps(10**(-5), int(1e6))
W_many = E_cost(1000, t_points4, cpl_period = 1/5, coupling = 'linear')

In [53]:
def plot_E_cost(E_costs):
    '''Plots a histogram of the works done over early specified amount of trajectories'''
    plt.figure()

    num_bins = 40
    n, bins, patches = plt.hist(E_costs, num_bins)
    plt.xlabel('W (kT)')
    plt.ylabel('counts')
    plt.title('Energy cost of measurement')
plot_E_cost(W_many[-1])
print('mean:', np.average(W_many[-1]))
print('std :', np.std(W_many[-1]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

mean: 1.0709367609644929
std : 1.5586458486974606


In [54]:
print('On average for each time step dt the work done on the system is:',-U_int_min/E_unit/20000)

On average for each time step dt the work done on the system is: 0.1524075850291405


The above number expressed in kT/2 units and for a coupling period of 20000 steps tells us that if we were to ommit one time step contribution to the final work done it would on average decrease by 0.3. Yeah so for dt = 10^(-10) the average energy cost of measurement is 0.4 in contrast to 0.1 for dt = 10^(-11) but calculated over the same time period with higher number of steps, 10 times more steps. This is still odd because it is not that we removed one timestep and the energy cost became less by the 0.3 difference but instead we icnceased the accuracy of the energy calcualted over the same time interval by increasing the nubmber of steps taken, does that suggests the accuracy of the first dt steps was not high enough, if so then how far can we push this? In a case of infinitesimal dt would the average energy cost of the measurement become 0? Also why is not zero in the first place, is that because we take more time steps during decoupling period than the coupling period?

What if we just decrease the dt and make the time interval smaller, thus making the coupling time shorter, would that give us higher mean? 

In [55]:
w_many_10 = E_cost(1000, t_points4, cpl_period=0.02)

In [56]:
plot_E_cost(w_many_10[-1])
print('mean:', np.average(w_many_10[-1]))
print('std :', np.std(w_many_10[-1]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

mean: 0.3017091657688983
std : 1.7679672117014495


## Quasistatic coupling

In [57]:
cpl_periods = np.arange(0.05, 1, 0.08)
cpl_periods_long = np.concatenate( (np.arange(0.01,0.13,0.01), np.arange(0.1, 1, 0.04)) )

@njit(parallel = True)
def E_cost_quas_couple(n, t_points, pdf = 'previous', couple_periods = cpl_periods):
    '''Energy cost of coupling averaged over n trajectories/runs for different coupling periods couple_periods, instantaneous decoupling
    Parameters
    ----------
    couple_periods: numpy array containing the coupling periods for which to calcualte the E_cost
    '''
    E_cost_avg = np.empty(couple_periods.size) 
    w_cpl_avg = np.empty(couple_periods.size) 
    w_dcpl_avg = np.empty(couple_periods.size) 
    
    for i in prange(couple_periods.size):
        w_cpl, w_dcpl, E_costs =  E_cost(n ,t_points, cpl_period = couple_periods[i], pdf = pdf) 
        E_cost_avg[i] = np.mean(E_costs)
        w_cpl_avg[i] = np.mean(w_cpl)
        w_dcpl_avg[i] = np.mean(w_dcpl)
        
    return w_cpl_avg, w_dcpl_avg, E_cost_avg

E_quasi = E_cost_quas_couple(100, t_points4, pdf = 'previous', couple_periods = cpl_periods_long)

In [58]:
plt.figure(figsize=(21,5))
plt.subplot(1,3,1)
plt.plot(cpl_periods_long, E_quasi[-2], 'or')
plt.title('⟨W⟩ decouple', fontsize= 20)
plt.ylabel('E (kT)', fontsize= 15)
plt.xlabel('$t_c/t_f$', fontsize= 15)
plt.ylim(3048,3049)
plt.grid()

plt.subplot(1,3,2)
plt.plot(cpl_periods_long, E_quasi[-3], 'og')
plt.title('⟨W⟩ couple', fontsize= 20)
plt.xlabel('$t_c/t_f$', fontsize= 15)
plt.grid()

plt.subplot(1,3,3)
plt.plot(cpl_periods_long, E_quasi[-1] ,'o', c='darkorange')
plt.xlabel('$t_c/t_f$', fontsize= 15)
plt.ylabel('$ ⟨W_{cost}⟩$ (kT)', fontsize= 15)
plt.title('Non-equilibrium demon', fontsize= 20)
plt.grid()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [59]:
plt.figure()

plt.plot(cpl_periods_long, E_quasi[-1] ,'o', c='darkorange')
plt.xlabel('$t_c/t_f$')
plt.ylabel('$ ⟨W_{cost}⟩$ (kT)')
plt.title('Non-equilibrium demon')
plt.grid()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

So I have already shown that the energy cost of measurement is accurately enough computed with the dts that we have used here, well I showed it for continuous coupling and decoupling but not for instantaneous decoupling but anyway the I have used t_points4 here and took an average over 1000 runs and still the average energy cost does go down for smaller coupling periods which is opposite of what we expected. This pretty much means that the faster we couple the system with the demon the more energy we are getting during the coupling period, if  we assume that the decoupling period's change in energy remains the same on average. That is strange, because a more quasistatic process should be able to get more energy out of the system during coupling and not the other way around. I still has no idea why are we seeing these results, it is not caused by insufficient accuracy, nor wrong specifications of the coupling and decoupling periods, or a mistake in the functions that calculate the work and solve the differential equations. I am going to leave this again and come back to this problem later, my understanding of the energy transfer processes is still very limited so I definitely want to read the Ian's section on the free energy and related topics so I can deal with this better.

In [60]:
cpl_periods1 = np.arange(0.0001, 0.01, 0.0005)
E_cost_average1 = np.empty(cpl_periods1.size)
print(cpl_periods1)

#for i in range(cpl_periods1.size):
#    E_cost_average1[i] = np.average( E_cost(1000,t_points4, cpl_period = cpl_periods1[i]) )

[0.0001 0.0006 0.0011 0.0016 0.0021 0.0026 0.0031 0.0036 0.0041 0.0046
 0.0051 0.0056 0.0061 0.0066 0.0071 0.0076 0.0081 0.0086 0.0091 0.0096]


In [61]:
plt.figure()

plt.plot(cpl_periods1, E_cost_average1/E_unit, 'x')
plt.xlabel('coupling period')
plt.ylabel('average energy cost')
plt.title('Quasistatic coupling')
plt.grid()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Instantaneous decoupling 
$$
\Delta W = \Delta E \\
= \Delta U + \Delta E_{kinetic} \\
= \Delta U
$$
Quasistatic decoupling
$$
\Delta W = \Delta F \\
= \Delta E - T\Delta S \\
= \Delta U - T\Delta S \\
$$
Non quasistatic decoupling 
$$
\Delta W = \Delta F + T\Delta S_i\\
= \Delta E - T\Delta S + T\Delta S_i \\
= \Delta U - T\Delta S + T\Delta S_i\\
$$

Before we reasoned that the energy cost of a measurement is zero on average for the two cases of quasistatic coupling and decoupling and when we abruptly couple and decouple the demon from syste, but for the latter it is only true if the demon is in a non equilibrium state and is hovering right above the global potential minima. Then we would be able to recover all the lost energy from immediate decoupling, but this is not the case when demon is initially in equilibrium and the microstate probability that describes the demon's state is the boltzman's distribution in which case the demon's position likelyhood would be evenly distributed over all the possible angle orrientations. If this was so than the average potential difference gained through coupling would be much smaller than the potential difference during dceoupling because the demon would never drop to the bottom immediately. This is the reason why we were recovering more energy during coupling for shorter coupling periods, we were in fact using the knowledge of the demon's non equilibrium state in addition to the measurement of system's angle. Alright, so the best way of operating this engine appears to be immediately decouple, immediately extract energy upon measurement and again immeadiately couple the system with demon, since all these 3 processes happend almost instantaneously the cost of decoupling is balanced by the energy gained through coupling and in addition to that we are also extracting energy out of the system upon the measurement. Well actualy, if we extract the energy out of the system by shifting the potential than that will change the minimum energy state for the system demon and which will has an effect of not droping the demon straight to its previous potential state but maybe slightly higher and the energy recovered from that might exactly compensate for the energy extracted through measurement.

In [62]:
sol_test = langevin_numba(init, t_points4, cpl_period = 1/5, dcpl_period = 1/5)

In [63]:
plot_correlation(sol_test,t_points4)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Quasistatic decoupling

In [64]:
@njit(parallel=True)
def E_cost_quas_dcpl(n, t_points, pdf = 'previous', decouple_periods = cpl_periods):
    '''Counterpart to E_cost_quasi_cpl() function but considers different decoupling times and the coupling period is fixed to 1/5 of tf'''
    E_cost_avg = np.empty(decouple_periods.size) 
    w_cpl_avg = np.empty(decouple_periods.size) 
    w_dcpl_avg = np.empty(decouple_periods.size) 
    
    for i in prange(decouple_periods.size):
        w_cpl, w_dcpl, E_costs =  E_cost(n, t_points, cpl_period = 1/5, dcpl_period = decouple_periods[i], pdf = pdf) 
        E_cost_avg[i] = np.mean(E_costs)
        w_cpl_avg[i] = np.mean(w_cpl)
        w_dcpl_avg[i] = np.mean(w_dcpl)
        
    return w_cpl_avg, w_dcpl_avg, E_cost_avg
dcpl_t = np.arange(0.01, 3/5,0.02)
E_dcpl_quasi = E_cost_quas_dcpl(100, t_points4, pdf = 'previous', decouple_periods = dcpl_t)

In [65]:
plt.figure(figsize=(21,5))
plt.subplot(1,3,1)
plt.plot(dcpl_t, E_dcpl_quasi[-2], 'or')
plt.title('⟨W⟩ decouple', fontsize= 20)
plt.ylabel('E (kT)', fontsize= 15)
plt.xlabel('$(t_f-t_d)/t_f$', fontsize= 15)
plt.grid()

plt.subplot(1,3,2)
plt.plot(dcpl_t, E_dcpl_quasi[-3], 'og')
plt.title('⟨W⟩ couple', fontsize= 20)
plt.xlabel('$(t_f-t_d)/t_f$', fontsize= 15)
plt.grid()

plt.subplot(1,3,3)
plt.plot(dcpl_t, E_dcpl_quasi[-1] ,'o', c='darkorange')
plt.xlabel('$(t_f-t_d)/t_f$', fontsize= 15)
plt.ylabel('$ ⟨W_{cost}⟩$ (kT)', fontsize= 15)
plt.title('Non-equilibrium demon', fontsize= 20)
plt.grid()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [66]:
plt.figure()

plt.plot(dcpl_t, E_dcpl_quasi[-1] ,'o', c='darkorange')
plt.xlabel('$(t_f-t_d)/t_f$')
plt.ylabel('$ ⟨W_{cost}⟩$ (kT)')
plt.title('Non-equilibrium demon, Energy cost of a measurement')
plt.grid()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Qusistatic coupling and decoupling

In [67]:
@njit(parallel = True) 
def E_cost_cdpl(n, t_points, pdf = 'previous'):
    '''E cost when both the coupling and decoupling are changed simultaneously'''
    E_cost_quas_cd = np.empty(cpl_periods.size)

    for i in prange(cpl_periods.size):
        E_cost_quas_cd[i] = np.mean( E_cost(n, t_points, cpl_period = cpl_periods[i], dcpl_period = cpl_periods[i], pdf = pdf)[-1] )
    return E_cost_quas_cd

E_cost_cdpl_previous = E_cost_cdpl(100, t_points4, pdf = 'previous')

In [68]:
plt.figure()

plt.plot(cpl_periods, E_cost_cdpl_previous/E_unit, 'x')
plt.xlabel('coupling|decoupling period')
plt.ylabel('average energy cost')
plt.title('Quasistatic coupling and decoupling')
plt.grid()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

I do not know how should i feel about this, that for smaller dt (10^(-11)s) the average energy cost of a measurement is much smaller than for bigger dt (10^(-10)), we kept the time intrval over which the equations of motion were solved the same, only thing that changed was number of steps which increased by a factor of 10 and dt decrease by a factor of 10 so the numerical accuracy of solving the equations and the energy cost of measurement went up. 

Hm what we get above seems almost alright, for longer time periods the energy cost of measurement is finally coming down but it drops below zero and the graphs suggests that it would continue on decreasing if we made the periods even longer, not sure about this. If it is just a coincidence that the datapoints in the graph just so happen to be negative with a negative slope. I am not quite getting what is it that made the trend shift from decreasing to decreasing energy cost of measurement in comparison to the instant decouple scenario. One thing it tells us is that decoupling continuously is less energeticaly expansive than an instantaneous shift in potential energy. That is just something that I do not understand and probably has to do with the entropy and free energy concepts that I am not fully familiar with. Okay I accidently plotted on more graph below with the same dt steps and it still gives us energies that are below 0 for high coupling periods, why is that?

In [69]:
cpl_periods_cd = np.arange(0.0001, 0.025, 0.0005)
E_cost_quast_cd1 = np.empty(cpl_periods_cd.size)
print(cpl_periods_cd)

#for i in range(cpl_periods_cd.size):
#    E_cost_quast_cd1[i] = np.average( E_cost(100, t_points4, cpl_period = cpl_periods_cd[i], dcpl_period = cpl_periods_cd[i]) )

[0.0001 0.0006 0.0011 0.0016 0.0021 0.0026 0.0031 0.0036 0.0041 0.0046
 0.0051 0.0056 0.0061 0.0066 0.0071 0.0076 0.0081 0.0086 0.0091 0.0096
 0.0101 0.0106 0.0111 0.0116 0.0121 0.0126 0.0131 0.0136 0.0141 0.0146
 0.0151 0.0156 0.0161 0.0166 0.0171 0.0176 0.0181 0.0186 0.0191 0.0196
 0.0201 0.0206 0.0211 0.0216 0.0221 0.0226 0.0231 0.0236 0.0241 0.0246]


In [70]:
plt.figure()

plt.plot(cpl_periods_cd, E_cost_quast_cd1/E_unit, 'x')
plt.xlabel('coupling|decoupling period')
plt.ylabel('average energy cost')
plt.title('Quasistatic coupling and decoupling')
plt.grid()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Energy cost of an exponential coupling
Testing how the energy cost of measurement changes when we use exponential coupling instead of linear.

In [71]:
w_many_exp = E_cost(1000, t_points4, cpl_period = 1, coupling = 'exp')

In [72]:
plot_E_cost(w_many_exp[-1])

print('mean:', np.average(w_many_exp[-1]))
print('std :', np.std(w_many_exp[-1]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

mean: 0.9164946217253467
std : 1.4092002633027014


So the coupling period was set to 1 which means that at 1/5 of the total time the coupling is about 86% and at time 1/10 is about 64% which is pretty close to what it would be if we linearly coupled the demon with system (100% and 50% respectively). The energy cost for exponential coupling appears to be smaller, mean of 1.3 whereas in the linear case the mean was about 1.6 but it is hard to make a direct comparison between these two methods of coupling. I might get back to cosidering different methods of coupling later when it might make a difference in some of these statistical quantities.

## initialized equilibrium pdf of system and demon before coupling 

In [73]:
w_many_eq = E_cost(1000, t_points4, cpl_period = 1/5, pdf = 'eq')

In [107]:
plot_E_cost(w_many_eq[-1])
print(np.average(w_many_eq[-1]))
print(np.std(w_many_eq[-1]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

1470.8305682877372
1354.5818279725047


In [75]:
w_many_eq2 = E_cost(1000, t_points4, cpl_period = 1/20, pdf = 'eq')

In [76]:
plot_E_cost(w_many_eq2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Quasistatic coupling and immediate decoupling

In [77]:
E_cost_quasi_eq = E_cost_quas_couple(1000, t_points_1, pdf = 'eq', couple_periods =  cpl_periods_long)

In [78]:
plt.figure()

plt.plot(cpl_periods_long, E_cost_quasi_eq[-1], 'o', c='darkorange')
plt.xlabel('$t_c/t_f$')
plt.ylabel('$ ⟨W_{cost}⟩$ (kT)')
plt.title('thermalized demon')
plt.grid()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Quasistatic coupling and decoupling 

In [79]:
E_cost_cdpl_eq = E_cost_cdpl(1000, t_points_1, pdf = 'eq')

In [80]:
plt.figure()

plt.plot(cpl_periods, E_cost_cdpl_eq/E_unit, 'x')
plt.xlabel('coupling|decoupling period')
plt.ylabel('average energy cost')
plt.title('Quasistatic coupling and decoupling')
plt.grid()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# E gained per cycle 
This section contains a rewritten code from the above section and does almost the same thing, calculates the energy cost of a measurement, energy extracted through exploitation and the total energy gained per one operational cycle.

Defining a langevin_wait that evolves the magnet's angles after they get decoupled but the system is still in contact with the coil.

In [81]:
@njit
def langevin_wait(r,  t_points):
    ''' Calculates the right hand sight of diff eqs
    r is a vector containing r = [[theta_s, w_s],
                                  [theta_d, w_d]]
    cpl_period : the fraction of time it takes to couple the demon to the system, originaly 1/5, default is the original
    '''
    # initializing empty arrays that will store the resulting trajectories
    theta_s = np.empty(t_points.shape)
    theta_d = np.empty(t_points.shape)
    w_s = np.empty(t_points.shape)
    w_d = np.empty(t_points.shape)
    
    # t final
    t_f = t_points[-1]
    h = t_f/(len(t_points)-1)
    # storing the initial values
    theta_s[0] = r[0,0]
    theta_d[0] = r[1,0]
    w_s[0] = r[0,1]
    w_d[0] = r[1,1]

    # solving the dif-eq
    for idx in range(1,len(t_points)):
        t = t_points[idx]
        # abreviation for variables at t
        th_s_t =  theta_s[idx-1]
        th_d_t = theta_d[idx-1]
        w_s_t = w_s[idx-1]
        w_d_t = w_d[idx-1]
        # calculate the variables at t+dt
        theta_s[idx] = th_s_t + h*w_s_t
        theta_d[idx] = th_d_t + h*w_d_t
        w_s[idx] = (w_s_t - h*S2*np.sin(th_s_t) -
                            h*S3*w_s_t +
                            h**(1/2)*S4*np.random.normal()  
                    )
        w_d[idx] = (w_d_t - h*D2*w_d_t  +
                            h**(1/2)*D3*np.random.normal()  
                   )
        
    sol = np.empty((2,2, t_points.shape[0]))
    sol[0,0] = theta_s
    sol[0,1] = w_s
    sol[1,0] = theta_d
    sol[1,1] = w_d
    return sol

In [82]:
@njit()
def E_per_cycle(n, t_points, cpl_period = 1/5, dcpl_period = 0, extraction = 'normal', wait = 0, pdf = 'previous', coupling = 'linear'):
    '''
    '''
    E_costs = np.empty(n)    # Energy cost of making the measurement
    E_exploit = np.empty(n) # Energy extracted out of the system
    
    w_couples = np.empty(n)
    w_decouples = np.empty(n)
    
    # constants
    A = mu_d_max*mu_s*u_0/(4*np.pi*d**3)
    C = B_0*mu_s
    t_f = t_points[-1]
    N = int(t_points.size - 1)
    
    # initial condition
    init_r = init
    
    for run in range(n):
        solution = langevin_numba(init_r, t_points, cpl_period = cpl_period, dcpl_period = dcpl_period, coupling = coupling)
        # demon's and system's final angles
        th_d_f = solution[1,0][-1] # final theta of demon
        th_s_f = solution[0,0][-1]
        
        # energy it takes to couple the system and demon
        E_couple   = work_couple_numba(solution, t_points, cpl_period = cpl_period, coupling = coupling) # energy it takes to couple 
        E_decouple = work_decouple(solution, t_points, dcpl_period = dcpl_period, coupling = coupling)   # energy it takes to decouple
        E_costs[run] = E_couple + E_decouple                          # energy cost of the measurement  
        
        w_couples[run] = E_couple
        w_decouples[run] = E_decouple
        # Energy extraction right after decoupling
        if (extraction == 'normal'): 
            # estimated/measuerd th_s by demon's angle
            th_s_meas = np.arctan(2*np.sin(th_d_f)*A / (C-A*np.cos(th_d_f)))
            # energy that goes out of the system
            E_exploit[run] = U_system(th_s_meas - th_s_f) - U_system(th_s_f)              
            
        # ideal extraction in a case of a perfect measurement     
        elif (extraction == 'perfect'):    
            E_exploit[run] = U_0_min - U_system(th_s_f) 
        
        # wait after the decouple.
        elif (extraction == 'wait'):
            N_wait = int(N*wait) # number of steps executed after decoupling
            t_wait = np.arange(0, t_f*wait + t_f*wait/N_wait, t_f*wait/N_wait) # time during the wait period

            sol_wait = langevin_wait(solution[:,:,-1], t_wait) # evolving the magnet's angles after they are decoupled
            
            th_s_f, th_d_f = sol_wait[0,0][-1], sol_wait[1,0][-1] # actual, system and demon's final angles
            th_s_meas = np.arctan(2*np.sin(th_d_f)*A / (C-A*np.cos(th_d_f))) # system's angle as measured by the demon
            E_exploit[run] = U_system(th_s_meas - th_s_f) - U_system(th_s_f) 
            
        #Specifying the initial angles and velcoities for the next run
        # using the angles and velocites from previous run (instantaneous decoupling and extraction of energy)
        if (pdf == 'previous'):
            init_r = solution[:,:,-1]
        # angles randomized according to boltzman equilibrium distribution and the velocities set to zero, np.random.choice not recognized by numba
        elif (pdf == 'eq'):
            init_r = np.zeros((2,2))
            th_d = np.random.random()*np.pi*2 - np.pi
            th_s = np.random.normal()*pdf_s_std
            init_r[0,0], init_r[1,0] = th_s, th_d
            
        # all zeros, zero velocities and angles at their respective minima
        elif (pdf == 'zero'):
            init_r = np.zeros((2,2))
            init_r[1,0] = np.pi
            
    return E_costs, E_exploit

## Instantaneous decoupling

In [83]:
E_costs, E_exploit = E_per_cycle(1000, t_points_1, cpl_period = 1/5, extraction = 'normal')
E_gain_n = E_exploit + E_costs
print(np.average(E_costs)/E_unit, np.average(E_exploit)/E_unit)

0.9704883664651413 -0.727985857764659


In [84]:
def plot_E_total(E_total):
    plt.figure()

    n, bins, patches = plt.hist(E_total/E_unit, 30)
    plt.xlabel('E_total')
    plt.ylabel('counts')
    plt.title('total energy on each cycle')
plot_E_total(E_gain_n)
print('Energy gained per 1 operational cycle\nmean:',np.mean(E_gain_n)/E_unit,'\nstd  :',np.std(E_gain_n)/E_unit)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Energy gained per 1 operational cycle
mean: 0.24250250870048232 
std  : 1.1427074685845977


In [85]:
E_costs1, E_exploit1 = E_per_cycle(1000, t_points_1, cpl_period = 1/5, extraction = 'perfect')
E_gain_n1 = E_exploit1 + E_costs1

In [86]:
plot_E_total(E_gain_n1)
print('Energy gained per 1 operational cycle\nmean:',np.mean(E_gain_n1)/E_unit,'\nstd  :',np.std(E_gain_n1)/E_unit)
print('E_exploit:',np.average(E_exploit1)/E_unit)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Energy gained per 1 operational cycle
mean: -0.1911830986233045 
std  : 1.391360636370925
E_exploit: -1.232121705165181


### Energy extracted through measurement exploitation
First the average energy of a perfect measurement is calculated.

In [87]:
E_costs_expl_p, E_exploit_expl_p = E_per_cycle(100, t_points4, cpl_period = 1/5, extraction = 'perfect')

In [88]:
E_expl_perfect = np.average(E_exploit_expl_p)/E_unit
print(E_expl_perfect)

-1.1094798258274012


In [89]:
E_costs_expl, E_exploit_expl = E_per_cycle(100, t_points4, cpl_period = 1/5, extraction = 'normal')

In [90]:
E_avg_expl = np.average(E_exploit_expl)/E_unit
E_errmean_expl = np.std(E_exploit_expl)/E_unit/np.sqrt(E_exploit_expl.size)
print('The average energy extracted <E_ext>={0:.3f} +/- {1:.3f}'.format(E_avg_expl,E_errmean_expl))
plt.figure()

plt.hist(E_exploit_expl/E_unit, 80, color='g')
plt.xlabel('$<E_{extract}>$ (kT)')
plt.ylabel('counts')
plt.xlim([-5,3])
plt.axvline(x= E_avg_expl, ymin=0, c='orange', linestyle="--")
plt.text(-1.05,17000, 'average  {:0.2f}'.format(E_avg_expl), rotation=90)
plt.title('Energy extracted upon measurement')

The average energy extracted <E_ext>=-0.827 +/- 0.171


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Energy extracted upon measurement')

## Continuous decoupling 
Nice results, nice, I tested how the energy change changes as the decouple period is made shorter. We started of 1/5 and that was too long, so long that there was no coupling correlation between the magnets left at the end of the process and the energy exploit on average was positive (energy lost during exploitation process). As the decoupling period was made shoter and we effectively decoupled faster the energy exploited and thus energy changed was getting smaller and smaller until it breached negative numbers. The point of the story is that we have to decouple fairly quickly if we want to maintain some coupling between the system and demon that could be exploited to extract positive energy out of the system. What is pleasing to see is that there is a trade-off between energy exploited and energy cost of measurement depending on the length of the decoupling period. The longer the period, the longer we wait and the less effective the demon's measurement is, but because we wait longer we waste less energy during the decoupling process. In reverse, it is desirable to immediately decouple the demon from the system because it would give us the most accurate measurement of system's angle and the energy exploit would be high, but a sudden decoupling is a non-quisistatic process which is very wastefull and the energy cost of measurement is bigger than for slower finite decoupling. This can be nicely visualised with a graph.

In [91]:
E_costs2, E_exploit2 = E_per_cycle(1000, t_points_1, cpl_period = 1/5, dcpl_period = 1/1000, extraction = 'normal')
E_gain_n2 = E_exploit2 + E_costs2
print(np.average(E_costs2)/E_unit, np.average(E_exploit2)/E_unit)

0.9752533594286605 -0.7433269614085294


In [92]:
plot_E_total(E_gain_n2)
print('Energy gained per 1 operational cycle\nmean:',np.mean(E_gain_n2)/E_unit,'\nstd  :',np.std(E_gain_n2)/E_unit)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Energy gained per 1 operational cycle
mean: 0.23192639802013107 
std  : 1.1448777593459478


In [93]:
E_costs3, E_exploit3 = E_per_cycle(1000, t_points_1, cpl_period = 1/5, dcpl_period = 2/5, extraction = 'normal')
E_gain_n3 = E_exploit3 + E_costs3
print(np.mean(E_costs3)/E_unit, np.average(E_exploit3)/E_unit)

0.8737471899251422 1.76401044486292


In [94]:
plot_E_total(E_gain_n3)
print('Energy gained per 1 operational cycle\nmean:',np.mean(E_gain_n3)/E_unit,'\nstd  :',np.std(E_gain_n3)/E_unit)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Energy gained per 1 operational cycle
mean: 2.6377576347880622 
std  : 3.248405809283578


## SLOW-IMMEDIATE decoupling trade off
Visualizing how the energy change, energy exploit and energy cost changes for different decoupling periods.

In [95]:
dcpl_periods = np.arange(0.01,0.2, 0.01)

@njit(parallel=True)
def quasi_dcpl_trade(n, pdf = 'previos'):
    E_cost_trade = np.empty(dcpl_periods.size)
    E_exploit_trade = np.empty(dcpl_periods.size)
    E_change_trade = np.empty(dcpl_periods.size)
    
    for dcpl in prange(dcpl_periods.size):
        E_cost_t, E_exploit_t = E_per_cycle(n, t_points_1, cpl_period = 1/5, dcpl_period = dcpl_periods[dcpl], pdf = pdf, extraction = 'normal')
        E_cost_trade[dcpl] = np.mean(E_cost_t)
        E_exploit_trade[dcpl] = np.mean(E_exploit_t)
        E_change_trade[dcpl] = E_cost_trade[dcpl] + E_exploit_trade[dcpl]
        
    return E_cost_trade/E_unit, E_exploit_trade/E_unit, E_change_trade/E_unit

#E_c_trade, E_exp_trade, E_change_trade = quasi_dcpl_trade(1000, pdf='previous')

In [96]:
plt.figure()

plt.plot(dcpl_periods, E_exp_trade ,label='E extract')
plt.plot(dcpl_periods, E_change_trade ,label='E total')
plt.plot(dcpl_periods, E_c_trade, label='E cost')
plt.xlabel('$(t_f-t_d)/t_f$')
plt.ylabel('E (kT)')
plt.grid()
plt.legend()
plt.title('Slow-immediate decoupling trade off')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

NameError: name 'E_exp_trade' is not defined

Aaaah beatiful, the exploitation energy start of by being useful and as the decoupling period is made longer the energy exploit flips to costing us energy instead of pumping energy out of the system. In contrast, the energy cost of measurement starts of high, being expensive becasue we non-quisistaticly decouple the magnets and for slower decoupling that cost goes down. Nevertheless the energy change which is the total energy gained/lost per engine's 1 operational cycle steadily  keeps increasing. If the dceoupling period was made even shorter we would get closer to zero net energy. In the case of a perfect measurement the net energy would cross the zero line into negative numbers and we would see a succesful operation of an engine. What might be of interest next is the increase of maximum coupling between the magnets, which should improve the performance of the engine and hopefuly give negative change in energies per cycle.

## Energy gained of a delayed measurement
In this section we will explore the changes in energy gained through measurement exploitation and total energy gained from the engine's operation per cycle as we delay the measurement of the system's angle after the magnets become completely decoupled. This means that we will wait some time after the magnets are fully decoupled and then we take the measurement of the system's angle and extract its energy out by shifting the coil's orientation. On average we would expect lower energy gains from exploitation for longer waiting periods.

We would like to probe how the energy cost of coupling changes as we slow down the turning on of the magnet's magnetic moment making it closer to a quasistatic process. It is expected to see a decrease in the cost because less heat should be transfered during the process, that is what the theory it is not something that I personaly understand. 

In [ ]:
wait_times =  np.linspace(10**(-3), 0.03, 10) # fraction of 1 cycle time we wait after decoupling
E_costs_many = np.empty(wait_times.shape)                    # Energy costs of measurement
E_exploit_many = np.empty(wait_times.shape)

for run in range(wait_times.size):
    E_costs, E_exploit = E_per_cycle(1000, t_points_1, cpl_period = 1/5, extraction = 'wait', wait = wait_times[run]) 
    E_costs_many[run] = np.average(E_costs)/E_unit
    E_exploit_many[run] = np.average(E_exploit)/E_unit

In [ ]:
E_gained_many = E_costs_many + E_exploit_many
plt.figure()

plt.plot(wait_times, E_exploit_many, label = '<E exploit>')
plt.plot(wait_times, E_gained_many, label = '<E gained>')
plt.plot(wait_times, E_costs_many, label = '<E meas. cost>')
plt.xlabel("waited time fraction")
plt.ylabel('E (kT)')
plt.title('Energy transfers after a delayed measurement')
plt.grid()
plt.legend()

The figure shows how the energy acquired through exploitation period and the total gained energy per cycle changes as we delay time of a measurement after the magnets are decoupled. First thing to notice is that the energy cost of a measurement stays unchanged as it is unaffected by delaying of the measurement. The blue line which represents the average energy extracted from the system upon measuring approaches zero and then it surpasses to positive numbers with a steady slope. After the time it crosses 0 the demon's measurement is no more useful to us. What is not obvious at first is the ever increasing energy extracted upon the measurement, so not only that we do not extract positive energy out of the system but we waste the energy by trying to aling the coil with the system magnet. In the case of magnets in harmonic potential, their movement was restricted to oscillations around the equilibrium points so I would guess that the after the oscillators became completely independent of each other, the measurement would be a completely random guess of the system's position and the energy extracted on average would be zero or, negative (meaning that we waste enregy by trying to gain energy, opposite of the sign convention above). But here with the magnets we see that as we wait longer the energy we waste by acting upon the demon's measurement is getting more and more positive (we loos energy instead of gain energy). This is because once the magnets stop interacting, the demon's motion becomes unconstrained and it goes on to wander over its entire range of allowed angles, from -pi to pi, whereas the system's motion is still modulated by the the coil's magnetic field, confining it to move around its equilibrium point and not deviate much from that. Since we were using the minimum of the interaction potential energy to measure the system's angle, the measurement would approximately tell us that the system is on the other side of the demon, but it really isnt because the demon when far out while the system stayed where it was before. Therefore this energy loss from exploitation period will be getting bigger and bigger because the demon will go futher and futhre away from the system's bottom of the potential well. If we ran the simulations for long enough (long wait time) we would eventually hit a constant average energy lost per cycle upon the measurement.